In [ ]:
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt

path = Path("/content/spectrogram")
counts = {p.name: len(list(p.glob("*.png"))) for p in path.iterdir() if p.is_dir()}

for k, v in counts.items():
    print(f"Class '{k}': {v} images")

plt.bar(counts.keys(), counts.values())
plt.title("Number of Images per Class")
plt.xlabel("Class")
plt.ylabel("Count")
plt.show()

In [ ]:
import os
import random
import shutil
from pathlib import Path

def split_dataset(
    source_dir='all_spectrograms',
    target_dir='dataset_split',
    train_ratio=0.7,
    valid_ratio=0.2,
    test_ratio=0.1,
    seed=42
):
    random.seed(seed)
    source_dir = Path(source_dir)
    target_dir = Path(target_dir)

    for split in ['train', 'valid', 'test']:
        for class_dir in (source_dir.iterdir()):
            (target_dir / split / class_dir.name).mkdir(parents=True, exist_ok=True)

    for class_dir in source_dir.iterdir():
        if not class_dir.is_dir():
            continue
        images = list(class_dir.glob('*.png'))
        random.shuffle(images)

        total = len(images)
        train_cutoff = int(train_ratio * total)
        valid_cutoff = train_cutoff + int(valid_ratio * total)

        for i, img_path in enumerate(images):
            if i < train_cutoff:
                split = 'train'
            elif i < valid_cutoff:
                split = 'valid'
            else:
                split = 'test'

            dst = target_dir / split / class_dir.name / img_path.name
            shutil.copy(img_path, dst)

   split_dataset(
    source_dir='/content/spectrogram',
    target_dir='dataset_split',
    train_ratio=0.7,
    valid_ratio=0.2,
    test_ratio=0.1
)

In [ ]:
from fastai.vision.all import *
import matplotlib.pyplot as plt
path = Path('/content/dataset_split')

dls = ImageDataLoaders.from_folder(
    path,
    train='train',
    valid='valid',
    item_tfms=Resize(224),
    batch_tfms=aug_transforms(),
    bs=32
)

learn = vision_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(5)
learn.export('chord_model.pkl')